In [ ]:
NAME = "" # put your full name here
COLLABORATORS = [] # list names of anyone you worked with on this homework.

# [ER 131] Homework 9: Model Selection and Regularization

This assignment explores methods that answer the question of how to choose which features to include in a model. In HW8 (resampling), we used cross-validation to evaluate how well different models generalize to new data, and one of the big takeaways was that using too many features can cause us to over-fit our model on the training data, and lead to a model that performs poorly on test data. In this HW, we'll look at two methods that attempt to reduce model variance by prioritizing certain model features: ridge regression, and lasso regression.

In this lab, we will be writing lots of functions. While these exercises take time and patience, you may find that the resulting functions are helpful in your final project (especially if you are working on a regression problem). 

A good reference for this HW is ISLR Ch. 6.

### Table of Contents

1. [The data](#data)<br>
1. [Comparing linear, ridge, and lasso regression](#models)<br>
1. [Standardizing your variables](#standardization)<br>
1. [Choosing lambda](#lambda)<br>
1. [Comparing optimal models](#compare)<br>

## Section 1: The Data<a name='data'></a>

We're going to be working (for the final time!) with the Novotny et al. LUR data. You will recognize it from HW6 and Lab 9. As a refresher:

* The data is an accumulation of GIS land-use characteristics from land-monitoring by the EPA and in situ NO2 measurements from satellite sensors.
* The goal of this land-use regression (LUR) is to estimate outdoor air pollution geospatially across the contiguous United States.
* The reason for the high number of data points is that the data keeps track of readings from monitors at a high resolution, up to ~30 meters.

First, let's upload the data to dataframe `df`.

In [ ]:
# Run this block

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set_context("talk")
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

np.random.seed(1)

pd.set_option('display.max_columns', None)

In [ ]:
# run this cell
df = pd.read_csv('data/BechleLUR_2006_allmodelbuildingdata.csv')
df.head(10)

In this lab, we're going to be creating models for two sets of data: one for the full dataset, and one for just California data. We'll be looking at how different approaches (lasso, and ridge) perform with datasets that have more or fewer observations.

**Question 1.1** To get started, create a dataframe `df_ca` that contains just California observations from dataframe `df`.

In [ ]:
df_ca = ... # YOUR CODE HERE

In [ ]:
assert df_ca.shape == (94,135)

**Question 1.2** As with the lab and previous HW, we want to predict Observed_NO2_ppb and we want to use all the columns except for Monitor_ID, State, Latitude, Longitude, Observed_NO2_ppb, and Predicted_NO2_ppb as features.

Write a function `get_X_y()` that takes as input a dataframe, and returns four dataframes: X_train, X_test, y_train, and y_test. Remember to start by selecting the X values (all the columns except for the ones listed above) and the one y value (observed NO2). 

In [ ]:
# solution
from sklearn.model_selection import train_test_split

def get_X_y(df, cols_to_drop, y_col, test_prop, rand_seed):
    """
    This function returns four dataframes containing the testing and training X and y values used in land-use regression.
    Input: df, a Pandas dataframe with all of the fields in the land-use regression dataset; 
        cols_to_drop, a list of the names (strings) of the columns to drop from df in order to obtain the feature variables.
        y_col, a column name (as a string) of df that represents the response variable
        test_prop, a float between 0 and 1 indicating the fraction of the data to include in the test split
        rand_seed, an integer, used to define the random state
    Returns: X_train, X_test, y_train, y_test, four dataframes containing the training and testing subsets of the 
    feature matrix X and response matrix y
    """
    
    # YOUR CODE HERE
    X = ...
    y = ...
    
    X_train, X_test, y_train, y_test = train_test_split(...)
    
    return X_train, X_test, y_train, y_test

**Question 1.3** Call your `get_X_y` function on `df` and `df_ca`, and save the resulting dataframes to `X_all_train, X_all_test, y_all_train, y_all_test` and `X_ca_train, X_ca_test, y_ca_train, y_ca_test` respectively.

Write your function to hold 25% of the data as test data, with a `random_state` of 2020 as input.  This is important!  The assert functions won't work if you don't use that random state.  

In [ ]:
# YOUR CODE HERE
X_all_train, X_all_test, y_all_train, y_all_test = ... # get X and y train and test dataframes for full dataset
                                                          
X_ca_train, X_ca_test, y_ca_train, y_ca_test = ... # get X and y train and test dataframes for CA data

In [ ]:
# check dimensions of output dataframes
assert X_ca_train.shape[0] == y_ca_train.shape[0]
assert X_all_test.shape[0] == y_all_test.shape[0] == 93
assert X_ca_test.shape[1] == X_all_test.shape[1] == 129

## Section 2: Comparing linear, ridge, and lasso regression<a name='models'></a>

Our feature matrix has a very high number of features in it (129 to be exact). It would be a big hassle to determine the importance of each independent variable (that would take ages!). So how should we go about indentifying which features to include in our model?

As a recap from Lab 9 and from lecture, for OLS regression, a model with two features would take a form that looks something like this:

$$\Large \hat{y} = \hat{\theta_1} x_{1i} + \hat{\theta_2} x_{2i}$$

However, when we have a large number of features (the number of features $p$ approaches the number of observations), OLS can result in high variance, leading to overfit. How would we fix this problem?

Enter **regularization.** We will begin with **ridge regression ($L^2$ regularization)**. This method finds the values of $\theta$ that minimize mean squared error plus a **regularization** or **penalty** term. Visually...

$$ \large \hat{\theta} = \arg \min_\theta \frac{1}{n} \sum_{i=1}^n \textbf{Loss}\left(y_i, \hat{y_i}\right) + \lambda {R_{L^2}}(\theta) $$

where $\large R_{L^2}(\theta) = \sum_{k=1}^p (\theta_k)^2$ and $p$ is the number of features.

In the case of Lasso regression, we would uses the penalty term $\large R_{L^1}(\theta) = \sum_{k=1}^p \Big|\theta_k\Big|$.

In both Ridge and Lasso regression, $\lambda$ is a hyperparameter that we can tune to balance the bias-variance tradeoff. The higher the value of $\large \lambda$, the more a model is penalized for including additional features. Essentially, we are decreasing variance and increasing bias with a large $\lambda$ value, and vice versa for a small $\lambda$ value.

For the purposes of `sklearn`, the lambda value will be passed in as an argument `alpha`.

What value of $\lambda$ would be lead to the right-sized penalty term? We need to check a few answers to find a good option for this term. What is a technique that we know that can will check for the "best fit" across different terms? You guessed it: **cross-validation**! We'll be doing cross validation later this notebook. For now, let's see how our models perform on a random train-test split.

**Question 2.1** In this homework we will compare the results of three models: simple linear regression, Ridge regression, and Lasso regression. As we know from Lab 9, `scikit-learn` has very generalizable syntax when it comes to using different models: regardless of which type of regression you are performing, you instantiate a model and save that instance of the model to a variable (eg. `lm = LinearRegression()`), then use `.fit()` to fit that model to the training data and `.predict()` to output predictions for test data.

In this question, we're going to complete a function `fit_model()` that automates the process of initializing a model, fitting that model, and getting predictions. The function should return two sets of values: the model coefficients and the test mean squared error of the model.

We've gotten you started with some skeleton code. It may be helpful to test out each part of the function separately, and then put it all together. It also may be helpful to look back at Lab 9 to see how we initialize and fit different models.

*Hint:* When getting the coefficients of a model using `.coef_`, sometimes scikit-learn returns a 1D array and sometimes it returns a nested array (i.e. with double brackets). To make sure that the dimensions of the coefficients that you output using this function are always the same, consider using the [numpy .flatten() method](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.flatten.html).

*Another hint:* We're fitting one of three potential models here: LinearRegression(), Ridge(), and Lasso(). While Ridge() and Lasso() take in an alpha argument, LinearRegression() does not! That means your code needs to check which type of model you'll be fitting before initializing that model and (if Lasso or Ridge) passing an alpha argument to it.

In [ ]:
# solution
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

def fit_model(Model, X_train, X_test, y_train, y_test, alpha = 1):
    """
    This function fits a model of type Model to the data in the training set of X and y, and finds the MSE on the test set
    Inputs: 
        Model (sklearn model): the type of sklearn model with which to fit the data - LinearRegression, Ridge, or Lasso
        X_train: the set of features used to train the model
        y_train: the set of response variable observations used to train the model
        X_test: the set of features used to test the model
        y_test: the set of response variable observations used to test the model
        alpha: the penalty parameter, to be used with Ridge and Lasso models only
    """    
    
    # initialize model
        
    # fit model
    
    # get mean squared error of test data
    mse = ...
    
    # get coefficients of model
    coef = ...
    
    return mse, coef

In [ ]:
# run this cell to confirm that fit_model() works as expected

# check the MSE of using Ridge regression on the full data - the value should be ~ 28 if your random_state was 2020 in get_X_y
assert 27 < fit_model(Ridge, X_all_train, X_all_test, y_all_train, y_all_test)[0] < 28

# make sure that the number of coefficients is equal to the number of features
assert len(fit_model(LinearRegression, X_all_train, X_all_test, y_all_train, y_all_test)[1]) == 129

# check that Lasso is reducing the coefficients as expected - i.e. by setting some coefficients to zero
assert np.count_nonzero(fit_model(Lasso, X_all_train, X_all_test, y_all_train, y_all_test)[1]) < len(fit_model(Lasso, X_all_train, X_all_test, y_all_train, y_all_test)[1])

**Question 2.2** For each set of data (i.e., the training and testing sets of `X_all, y_all` and `X_ca, y_ca` obtained in Section 1), call `fit_model()` 3 times: once with `Model = LinearRegression`, once with `Model = Ridge` and once with `Model = Lasso`. For Ridge and Lasso, you can leave the `alpha` value as its default value (1). Save both the MSE and the coefficient output everytime you run the models. You'll end up with 6 MSE values and 6 sets of coefficients (3 for the full dataset and 3 for California; 1 per model type in each case). It's up to you how you store the values (you can use lists or arrays or separate variable names, as long as you're able to access the MSE values and coefficients later in the lab). 

In [ ]:
# YOUR CODE HERE

**Question 2.3** Create a plot with two subplots below. In the first subplot, plot the coefficients for Ridge regression, Lasso regression, and simple linear regression for the full dataset. In the second, plot the same, but for the California data. Skeleton code is provided to construct a bar plot, but you can change this to use whatever type of plot makes the most sense to you. Make sure to have axis labels, titles, and legends.

In [ ]:
# replace ellipses with your code

# Set up the bar plot arrangement
ind = np.arange(coef_all.shape[0])
width = 0.25
pos = np.array([ind - width, ind, ind + width])
modelNames = ["Linear regression", "Ridge", "Lasso"]

plt.figure(figsize = (20,10))

# Plot the coefficients for Linear, Ridge, and Lasso models fit using all data
plt.subplot(211)
for i in np.arange(coef_all.shape[1]):
    plt.bar(x = ..., height = ..., width = ..., label = ...)
plt.legend()
plt.xlabel(...)
plt.ylabel(...)
plt.title(...)

# Plot the coefficients for Linear, Ridge, and Lasso models fit using CA data
plt.subplot(212)
for i in np.arange(coef_all.shape[1]):
    plt.bar(x = ..., height = ..., width = ..., label = ...)
plt.legend()
plt.xlabel(...)
plt.ylabel(...)
plt.title(...)

plt.tight_layout()
plt.show()

**Question 2.4** How do the coefficients from lasso, ridge, and linear regression compare to each other for the full dataset? How about for the California data? Based on your knowledge of lasso, ridge, and linear regression, can you explain why the coefficients compare in this way?

*YOUR ANSWER HERE*

**Question 2.5** Compare the 3 test MSES from the full dataset and from the California dataset. How do the MSEs compare within model types (e.g., how does the MSE for linear regression fit on the California model compare with the linear regression fit using all the data)? Can you explain how the two sets of values compare, based on what you know about bias and variance? 

Particularly, note how the MSEs change for the California data when Lasso is used vs. for all data when Lasso is used. Which set of data exhibits a larger improvement when Lasso is used? Why do you think that is?

You don't have to use a visualization for this question, although you are welcome to.

In [ ]:
# scratch work here

*YOUR ANSWER HERE*

## Section 3: Standardizing your variables<a name='standardization'></a>

As you may recall from lecture, it's a good idea to standardize your data when you are using regularization methods, especially when the domains of the independent variables span dramatically different ranges.

**Question 3.1** Use the `.describe()` method on the original `df` dataframe and examine the results. Provide at least two examples of features whose values span dramatically different ranges (i.e., at least one order of magnitude). In your own words (and referring to the asynchronous content as needed, describe why we should standardize our data when we are using regularization methods.

In [ ]:
# YOUR CODE HERE

*YOUR ANSWER HERE*

Fortunately for us, `sklearn` has built a built-in method, `StandardScaler` that makes it easy to standardize our variables before we fit our regression models. You can read more about it [here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html ). In essence, `StandardScaler` takes in our array of features. For each feature (i.e., column), the method takes the difference between each element and the mean value of the feature and divides the result by the standard devation of the features, according to the following equation:

$$
X_{stnd} = \frac{X - \mu}{\sigma}
$$

where $\mu$ and $\sigma$ are the mean and standard deviation, respectively, of the feature $X$. 

[Side note: Strictly speaking, we should only use `StandardScaler` if our features are normally distributed. If that's not the case, `sklearn` offers other scaling methods. For further reading, [see this article](https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02 ).]

The following code block shows an example of how to use `StandardScaler` to standardize the first three features from the Novotny et. al data.

In [ ]:
from sklearn.preprocessing import StandardScaler


X_test = df.iloc[:,6:9] # We'll standardize the WRF+DOMINO, Distance_to_coast_km, and Elevation_truncated_km features

scaler = StandardScaler() # Initialize the StandardScaler
scaler.fit(X_test) # Fit the standard scaler for each feature
print(scaler.mean_) # Print the mean value of each feature

X_stnd = scaler.transform(X_test) # Standardize each feature
X_stnd

**Question 3.2** Using the code above as a template, update your `get_X_y` function to standardize all of the features before splitting the data into testing and training sets. Then, call your new `get_X_y_stnd` function on `df` and `df_ca`, and save the resulting dataframes to `X_all_train, X_all_test, y_all_train, y_all_test` and `X_ca_train, X_ca_test, y_ca_train, y_ca_test` respectively. Again, use a test_size of 0.25 and random seed of 2020.

In [ ]:
def get_X_y_stnd(df, cols_to_drop, y_col, test_prop, rand_seed):
    """
    This function returns four dataframes containing the testing and training X and y values used in land-use regression.
    It standardizes the X variables prior to splitting the data.
    Input: df, a Pandas dataframe with all of the fields in the land-use regression dataset; 
        cols_to_drop, a list of the names (strings) of the columns to drop from df in order to obtain the feature variables.
        y_col, a column name (as a string) of df that represents the response variable
        test_prop, a float between 0 and 1 indicating the fraction of the data to include in the test split
        rand_seed, an integer, used to define the random state
    Returns: X_train, X_test, y_train, y_test, four dataframes containing the training and testing subsets of the 
    feature matrix X and response matrix y
    """
    
    # YOUR CODE HERE
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_all_train, X_all_test, y_all_train, y_all_test = ... # get X and y train and test dataframes for full dataset
X_ca_train, X_ca_test, y_ca_train, y_ca_test =  # get X and y train and test dataframes for CA data

In [ ]:
assert -1.4 < X_all_train[:,0].all() < 4.7
assert 0 < y_ca_test['Observed_NO2_ppb'].all() < 29

**Question 3.3** Replicate your process for Questions 2.2 and 2.3, but this time use your newly standardized data. Your result should be a plot with two subplots (one for the full dataset and the other for CA only), similar to the figure you produced in 2.3. 

In [ ]:
# YOUR CODE HERE to get the coefficients and MSE for each model and dataset

In [ ]:
# YOUR CODE HERE to plot the data

**Question 3.4** How do your coefficients change for linear regression, Ridge, and Lasso models when you standardize your features relative to when you do not? Can you explain these changes?

In [ ]:
#scratch work here

*YOUR ANSWER HERE*

## Section 4: Choosing lambda<a name='lambda'></a>

To further tune our model for maximum predictive power, we'll now use cross-validation to tune our hyperparameter $\lambda$.

In [ ]:
# run this cell
from sklearn.model_selection import KFold

**Question 4.1:** In this question, you'll complete a function `model_cv_mse()`. This function should perform k-fold cross validation on the training data, which is passed to the function through `X` and `y`, using the model (`Ridge()` or `Lasso()`; no LinearRegression this time) specified by the input `Model` for each value of alpha in the list `alphas`. It should then calculate the MSE associated with each alpha in `alphas`, and return a list of mean squared errors.

This time, use a `random_state` of 9 for `KFold`, and set `shuffle = True`.

This function will end up looking a lot like `mse_k_fold_lr()` from HW8.

In [ ]:
def model_cv_mse(Model, X, y, alphas, k = 3):
    """
    This function calculates the MSE resulting from k-fold CV using Lasso or Ridge regression performed on a training subset of 
    X and y for different values of alpha.
    Inputs: 
        Model (sklearn model): the type of sklearn model with which to fit the data - LinearRegression, Ridge, or Lasso
        X: the set of features used to fit the model
        y: the set of response variable observations
        alphas: a list of penalty parameters
        k: number of folds in k-fold cross-validation
    Returns:
        mses: a list containing the mean squared cross-validation error corresponding to each value of alpha
    """
    mses = ... # initialize array of MSEs to contain MSE for each fold and each value of alpha
        
    kf = KFold(...) # get kfold split
    
    fold = 0
    for train_i, val_i in kf.split(X):
        # get training and validation values
        X_f_train = ...
        X_f_val = ...
        y_f_train = ...
        y_f_val = ...
        
        for i in range(len(alphas)): # loop through alpha values
            model = ... # initialize model

            model.fit(...) # fit model
            
            y_pred = ... # get predictions
            
            # save MSE for this fold and alpha value
        
        fold += 1
    
    average_mses = ... # get average MSE for each alpha value across folds
    
    return average_mses

In [ ]:
# run this cell
alphas_test = [0.01, 0.1, 1, 10, 100, 1000, 10000]

# check dimensions of output MSE
assert len(model_cv_mse(Ridge, X_all_train, y_all_train, alphas_test)) == len(alphas_test)

# check values of output MSE at first alpha - should be about 10.4
assert 10 < model_cv_mse(Lasso, X_all_train, y_all_train, alphas_test)[0] < 11

# check reproducibility - the function should give the same MSE for the same alphas every time it runs
assert np.array_equal(model_cv_mse(Ridge, X_all_train, y_all_train, alphas_test),
                       model_cv_mse(Ridge, X_all_train, y_all_train, alphas_test))

**Question 4.2** Now that we can calculate the cross-validated MSE for different values of alpha, we can visualize the relationship between alpha and cross-validated MSE. Below, complete the skeleton code to create two subplots for the full dataset: the first subplot should show alpha vs. cross-validated MSE for Ridge regression for the training data, and the second should show alpha vs cross-validated MSE for Lasso regression for the training data. Use 5-fold CV.

It's up to you what range of alphas you want to show, but your plot should aim to roughly include the range around which the minimum MSE value appears. As a starting point, look at the output MSE values using the `test_alphas` values in the `assert` code block above, and try to decide which alphas to "zoom in" on from there - you may also decide to extend the range of alphas. It may take some trial and error.

*Hint*: You will end up with two very different ranges of alpha for lasso and ridge. 

Make sure to add axis labels and titles.

In [ ]:
alphas_ridge = ...
alphas_lasso = ...

plt.figure(figsize = (20,10))

plt.subplot(121)
plt.plot(...)
plt.title(...)
plt.xlabel(...)
plt.ylabel(...)

plt.subplot(122)
plt.plot(...)
plt.title(...)
plt.xlabel(...)
plt.ylabel(...)

plt.show()

 **Question 4.3** Comment on the plots above. How does the MSE change with different alphas for ridge vs lasso?

*YOUR OBSERVATIONS HERE*

**Question 4.4:** In what cases would you expect ridge regression to perform better than lasso, and vice versa? 

*YOUR OBSERVATION HERE*

## Section 5: Comparing optimal models<a name='compare'></a>

In this final section, we'll compare the performance of three models: your Lasso model with optimal alpha, your Ridge model with optimal alpha, and Novotny's model.

**Question 5.1:** Let's start by getting the optimal alpha and corresponding coefficients of the Ridge model. In this section, we'll use a new scikit-learn function, [`RidgeCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html). `RidgeCV()` takes as an input a list of alpha values and a cross-validation splitter object (which is just the output of the function `KFold(...)`).

To call `RidgeCV()`, you would want to first create a cross-validation object using `Kfold()`:

`kf = KFold(n_splits = ..., shuffle = ..., random_state = ...)`

You can then pass `kf` to `RidgeCV()`, along with a list of alpha values:

`ridgecv = RidgeCV(cv = kf, alphas = ...)`

and you can treat it essentially like any other `scikit-learn` model by calling `.fit()`:

`ridgecv.fit(X_train, y_train)`

Running the code above will, in one line, perform cross-validation, choose an ideal alpha from your list, and select the corresponding coefficients. You can then access the optimal alpha using:

`ridgecv.alpha_`

and the corresponding coefficients using:

`ridgecv.coef_`

You can also get predictions on your test data using `.predict()`:

`ridgecv.predict(X_test)`

In the cell below, we've written some skeleton code to get you started. You'll want to call `RidgeCV()` using 5-fold cross validation with a random state of 9, train the model on the training data for the **full** dataset (i.e. `X_all_train, y_all_train`), get predictions for the test data (i.e. `X_all_test`), and find the mean-squared-error on your cross-validated ridge model on the test data. Save the mean squared error to variable `ridgecv_mse`.

You may also want to print the optimal alpha value to make sure it's consistent with your plot in question 4.2.

You can use the same range of alphas that you did for plotting, or you can choose to use a smaller, larger, or more granular range.

In [ ]:
from sklearn.linear_model import RidgeCV

kf = KFold(...) # get KFold cross-validation selector object
ridgecv = RidgeCV(cv = ..., alphas=...) # pass CV object and list of alphas to RidgeCV()
ridgecv.fit(...) # fit RidgeCV model on training data

alpha_opt = ... # get optimal alpha value
print("optimal alpha:", alpha_opt)

y_pred_ridgecv = ... # get test predictions using RidgeCV model

ridgecv_mse = ... # get MSE of RidgeCV model
print("Test MSE with cross-validated Ridge:", ridgecv_mse)

**Question 5.2:** Now, let's get the optimal alpha and corresponding coefficients of the Lasso model. We'll use the function [`LassoCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html). As in question 5.1, `LassoCV()` takes as an input a list of alpha values and a cross-validation splitter object (which is just the output of the function `KFold(...)`). You would call `LassoCV()` and use it to fit a model, get predictions, and access the alpha and coefficient values in the same way you would `RidgeCV()`.

In this section, you'll want to call `LassoCV()` using 5-fold cross validation with a random state of 9, train the model on the training data for the **full** dataset (i.e. `X_all_train, y_all_train`), get predictions for the test data, and find the mean-squared-error on your cross-validated Lasso model on the test data. Save the mean squared error to variable `lassocv_mse`.

You may also want to print the optimal alpha value to make sure it's consistent with your plot in question 4.2. Note that it's not unusual if LassoCV() returns an optimal alpha that's slightly different than the alpha that shows up at the minimum point in your plot in question 4.2, due to how LassoCV() solves for coefficients. However, the alpha value should be within a reasonable range of the minimum alpha you get in question 4.2.

Again, you can use the same range of alphas that you did for plotting, or you can choose to use a smaller, larger, or more granular range.

In [ ]:
from sklearn.linear_model import LassoCV

kf = KFold(...) # get KFold cross-validation selector object
lassocv = LassoCV(cv = ..., alphas=...) # pass CV object and list of alphas to LassoCV()
lassocv.fit(...) # fit RidgeCV model on training data

alpha_opt = ... # get optimal alpha value
print("optimal alpha:", alpha_opt)

y_pred_lassocv = ... # get test predictions using RidgeCV model

lassocv_mse = ... # get MSE of LassoCV model
print("Test MSE with cross-validated Lasso:", lassocv_mse)

As an aside: at this point, you might be asking yourself why we wrote the function in Question 4.1 when we can use `RidgeCV()` and `LassoCV()`. While `RidgeCV()` and `LassoCV()` are very useful for getting optimal alphas and corresponding coefficients and mean squared errors, it's harder to extract the cross-validated mean squared error for a *list* of alphas from those functions (particularly from `LassoCV()`). We wanted you to have both tools (a self-written function that calculates MSE across alphas, and scikit-learn provided functions that return the optimal alpha and coefficients) at your disposal.

Moving on!

**Question 5.3** Now, let's see how Novotny's model performed. The list `X_nov` below are the predictors that the authors put in their final model data set.  Use that list to fit a multiple linear regression model whose predictors are just the predictors listed in `X_nov`.  Fit the linear regresion model to the training dataset we've been using throughout the lab (`X_all_train, y_all_train`) but select only the columns in `X_nov` from that training set. Then calculate the mean squared error between the model's prediction on the test data set (i.e. `X_test`, but containing only the columns in `X_nov`) and the actual data for the test set.  Save the MSE value to `novotny_mse`.

In [ ]:
# # Run this block; the numbers in X_nov correspond to the column names of the X_all datasets post standardization.


# X_nov = ['WRF+DOMINO',
#        'Impervious_800', 'Elevation_truncated_km', 'Major_800',
#        'Resident_100', 'Distance_to_coast_km']

X_nov = [0, 10, 2, 54, 69, 1]

In [ ]:
# YOUR CODE HERE
X_nov_train = ...
X_nov_test = ...

n...

print("Test MSE with Novotny model:", novotny_mse)

**Question 5.4** Which model performs best?  

*Your answer here*

**Question 5.5** In this question, we compared the MSE of 3 models. Name and describe one other metric we could use to compare models. How do you think the relative model performance of ridge, lasso, and Novotny's model would differ using that metric?

*YOUR ANSWER HERE*

----

## Submission

Congrats, you're done with homework 9!

Before you submit, click **Kernel** --> **Restart & Clear Output**. Then, click **Cell** --> **Run All**. Then, go to the toolbar and click **File** -> **Download as** -> **.html** and submit the file **as both an .html and .ipynb file through bCourses**.

----

## Bibliography

https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/



---
Notebook developed by: Alex Nakagawa, editted by Jessica Katz

Data Science Modules: http://data.berkeley.edu/education/modules
